# 02. NNLM (신경망 언어 모델): 의미를 이해하는 AI

## 1. N-gram의 한계와 신경망의 등장
앞서 배운 **N-gram**은 단순히 "통계(카운트)"에 의존했습니다. 그래서 본 적 없는 문장은 절대 맞출 수 없었죠.

반면, **NNLM(Neural Network Language Model)**은 단어의 **"의미(Meaning)"**를 학습합니다.

### 쉬운 비유: 암기왕 vs 이해왕
- **N-gram (암기왕)**: "'귀여운' 뒤엔 '고양이'가 나왔었지!" (통째로 외움)
- **NNLM (이해왕)**: "'고양이'랑 '강아지'는 비슷한 동물이네? 그럼 '귀여운' 뒤에 '강아지'도 올 수 있겠구나!" (응용력 발휘)

이 "의미"를 숫자로 표현한 것을 **임베딩(Embedding)**이라고 합니다.

## 2. 모델 구조 시각화

신경망이 단어를 예측하는 과정은 다음과 같습니다.

```
 [입력 단어들]      [의미 지도(Embedding)]        [생각(Hidden)]        [예측(Output)]
 "I", "love"  --->  [숫자 좌표로 변환]  --->  [정보를 섞고 처리]  --->  "NLP"
```

이제 이 과정을 코드로 하나씩 구현해 봅시다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

print("PyTorch 준비 완료")

PyTorch 준비 완료


## 3. 데이터 준비 (Data Preparation)

AI에게 가르칠 문장들을 준비하고, 컴퓨터가 이해할 수 있는 **숫자(Index)**로 바꿔줍니다.

In [2]:
# 1. 예시 문장
sentences = [
    "i like dog",
    "i like cat",
    "i like animal"
]

# 2. 단어 사전 만들기 (중복 제거)
word_list = " ".join(sentences).split()
word_list = list(set(word_list))

# 단어 -> 숫자 (Encoding)
word_dict = {w: i for i, w in enumerate(word_list)}
# 숫자 -> 단어 (Decoding)
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)  # 총 단어 개수

print("단어 사전:", word_dict)

단어 사전: {'animal': 0, 'dog': 1, 'cat': 2, 'like': 3, 'i': 4}


In [ ]:
# 3. 학습 데이터 만들기 (입력과 정답 분리)
# 목표: 앞의 2단어를 보고 -> 마지막 1단어 맞추기

input_batch = []
target_batch = []

for sen in sentences:
    word = sen.split()
    # 입력: 앞의 2개 (예: i, like)
    input = [word_dict[n] for n in word[:-1]]
    # 정답: 맨 뒤 1개 (예: dog)
    target = word_dict[word[-1]]

    input_batch.append(input)
    target_batch.append(target)

# 파이토치에서 쓸 수 있게 텐서(Tensor)로 변환
input_batch = torch.LongTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

print("입력 데이터:", input_batch)
print("정답 데이터:", target_batch)

## 4. 신경망 모델 만들기 (NNLM)

이제 뇌 구조(모델)를 설계합니다.

### 주요 부품 설명
1.  **Embedding (의미 지도)**: 단어 번호를 주면, 그 단어의 의미가 담긴 좌표(벡터)를 꺼내줍니다.
2.  **Linear (신경망 층)**: 꺼낸 의미들을 섞고 계산해서 패턴을 찾습니다.
3.  **Softmax (확률 변환)**: 계산 결과를 "확률"로 바꿔줍니다. (가장 높은 확률 = 정답 예측)

In [ ]:
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        
        # 1. 임베딩 층: 단어 개수(n_class)만큼의 단어를 2차원 좌표로 표현
        self.C = nn.Embedding(n_class, 2)
        
        # 2. 은닉 층(Hidden Layer): 입력된 2개 단어의 의미를 섞는 곳
        # 입력 크기 = 단어 2개 * 임베딩 크기 2 = 4
        # 출력 크기 = 내 마음대로 설정 (여기선 2)
        self.H = nn.Linear(2 * 2, 2)
        self.tanh = nn.Tanh() # 활성화 함수 (신호 조절기)

        # 3. 출력 층(Output Layer): 최종적으로 어떤 단어인지 예측
        self.U = nn.Linear(2, n_class)

    def forward(self, x):
        # x: [입력 단어 1, 입력 단어 2]
        
        # 1단계: 단어를 의미 벡터로 변환
        embeds = self.C(x) 
        # (Batch, 2개 단어, 2차원) -> (Batch, 4차원)으로 펼치기
        embeds = embeds.view(-1, 2 * 2)
        
        # 2단계: 신경망 통과 (생각하기)
        hidden = self.tanh(self.H(embeds))
        
        # 3단계: 최종 점수 계산
        output = self.U(hidden)
        return output

model = NNLM()
print("모델 설계 완료!")

## 5. 학습하기 (Training)

모델에게 문제를 풀게 하고, 틀릴 때마다 정답을 알려줘서 수정하게 합니다.
- **Loss (오차)**: 틀린 정도. (0에 가까울수록 좋음)
- **Optimizer (최적화 도구)**: 오차를 보고 모델을 고쳐주는 기술자.

In [ ]:
criterion = nn.CrossEntropyLoss() # 오차 계산기
optimizer = optim.Adam(model.parameters(), lr=0.01) # 수정 기술자

# 공부 시작!
for epoch in range(1000): # 1000번 반복 학습
    optimizer.zero_grad() # 이전 기억 지우기
    
    # 1. 문제 풀기 (예측)
    output = model(input_batch)
    
    # 2. 채점 하기 (오차 계산)
    loss = criterion(output, target_batch)
    
    # 3. 복습 하기 (모델 수정)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 200 == 0:
        print(f"{epoch+1}번 학습 완료... 오차(Loss): {loss.item():.4f}")

## 6. 테스트 (Test)

학습이 끝난 모델에게 질문을 던져봅니다.
"i like" 라고 말하면 그 뒤에 뭐가 올지 맞출 수 있을까요?

In [ ]:
# 예측해보기
predict = model(input_batch).data.max(1, keepdim=True)[1]

print("[결과 확인]")
for i in range(len(sentences)):
    in_word = sentences[i].split()[:2]
    true_word = sentences[i].split()[-1]
    pred_word = number_dict[predict[i].item()]
    
    print(f"입력: {in_word} -> 원래 정답: {true_word} | AI 예측: {pred_word}")
    
if predict[0].item() == target_batch[0].item():
    print("\n정답입니다! AI가 패턴을 익혔네요.")

## 7. 핵심 요약 퀴즈

1. **NNLM**이 N-gram과 다르게 '단어의 의미'를 저장하는 공간의 이름은 무엇인가요?
   - [ ] Counter Layer
   - [ ] Embedding Layer

2. 모델을 학습시킬 때 **Loss(오차)** 값은 어떻게 변해야 정상인가요?
   - [ ] 점점 커져야 한다.
   - [ ] 점점 작아져서 0에 가까워져야 한다.

정답을 생각해보며 코드를 복습해보세요!